In [1]:
import ee 
import geemap
import plotly.graph_objects as go
from ipyleaflet import WidgetControl
from ipywidgets import HTML

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-vl99956018')

pre_start_date = '2025-01-01'  # Período antes do fogo
pre_end_date = '2025-01-05'
pos_start_date = '2025-01-06'  # Período após o fogo
pos_end_date = '2025-01-16'

# Região de interesse (ROI)
roi = ee.Geometry.Point(-118.532875, 34.092615).buffer(15000)

rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3500,  # Valor único para todas as bandas
    'gamma': 1.2
}


ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

nbr_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'black', 'red']
}

In [ ]:
def get_collection(start_date, end_date):
    collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .select('B8', 'B12', 'B4', 'B3', 'B2')

    image = collection.median().clip(roi)
    return image